## MLflow architecture

### MLflow Components and Their Tasks

 - MLflow Project - creating an environment for experiments, grouping experiments.
 - MLflow Tracking - fixing parameters and quality metrics of experiments.
 - MLflow Models - preparing a version of the model for distribution.
 - MLflow Registry - centralized repository of models and layout in operation.

#### MLflow Projects

- Setting up the environment:
- programming languages
- package manager (e.g. conda)
- dependency (xgboost libraries, scikit-learn, ...)

- Description of the environment (Infrastructure as code):
- various OS
- local environment
- cloud services

#### MLflow Tracking

Fixes everything related to the launch of the model:
 - Datasets (for training and testing)
 - Sets of parameters (e.g. number of trees, layers, L1 / L2)
 - Values of quality metrics
 - Speed of work and other technical metrics.

#### MLflow Models

Serializes model artifacts as needed additional development

#### MLflow Registry

 - Centralized storage of model versions for easy search.
 - Information about which model is used on which environment
 - History of all versions and their use in environments

## 1. Working with a project and running an experiment

A project is a folder with files associated with this project:
  - File with MLProject metadata (YAML format) and also include files (for example, conda.yaml)
  - Files with code for running experiments (for example, in Python)

To create a project, it is enough to describe the correct MLProject file.

### Example of creating a project

The `MLproject` project file specifies that the model is trained in the` conda` environment, and is used by `scikit-learn` as the ML library (specified in` conda.yaml`). The training of the model itself is described in the file `train.py`, the necessary data preparation is also described there.

In [1]:
%%writefile MLproject
name: tutorial

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"

Overwriting MLproject


In [2]:
%%writefile conda.yaml
name: tutorial
channels:
  - defaults
dependencies:
  - numpy>=1.14.3
  - pandas>=1.0.0
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow

Overwriting conda.yaml


We are going to use [Wine Quality](https://github.com/mlflow/mlflow-example)

#### Preparing data for the experiment

The following code imports the modules necessary for work, loads the data from the `wine-quality.csv` file and splits them into test and validation selections

In [3]:
import os
import sys
import warnings
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

MLFLOW_SERVER_URL = 'http://127.0.0.1:5000/'

warnings.filterwarnings("ignore")
np.random.seed(40)
data = pd.read_csv("mlflow-example/wine-quality.csv")

train, test = train_test_split(data)

train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

### Create and run an experiment

The experiment code itself does not depend on MLflow, you can use the ready-made code.

To fix the launch parameters and metrics of the model, you need to start training within the experiment and project.

`tracking_url` - the address of the raised` mlflow` server, which will be used to store experiments. The web interface is also available at this address for viewing the launch results.

#### Running the experiment

To run the experiment, you need to execute the model creation code inside the MLflow launch context and store the parameters and resulting metrics in this context.

In [4]:
# connect to the server
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'experiment2'
mlflow.set_experiment(experiment_name)

# run the experiment
with mlflow.start_run():
    alpha = 0.5
    l1_ratio = 0.5

    # model
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    # metrics
    predicted_qualities = lr.predict(test_x)
    rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))
    mae = mean_absolute_error(test_y, predicted_qualities)
    r2 = r2_score(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # save the metric values for the experiment
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

INFO: 'experiment2' does not exist. Creating a new experiment
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284976
  MAE: 0.627876141016
  R2: 0.126787219728


## 2. Preparing the model for distribution

The launch of a successful experiment can be prepared for commissioning.

For this, the `MLflow Model Registry` is used.

### Review the conducted experiments and select the candidate for deployment

To get information about running experiments, you need to create a client `mlflow.tracking.MlflowClient`, then select the experiment you want and select the desired experiment start.

The code below takes the last run of the experiment from the list of all runs.

In [5]:
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
run_info = client.list_run_infos(experiment.experiment_id)[-1]

print(experiment)
print(run_info)

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='experiment2', tags={}>
<RunInfo: artifact_uri='./mlruns/1/0673d5e01497435ba313bd561c7f57ca/artifacts', end_time=1651779139325, experiment_id='1', lifecycle_stage='active', run_id='0673d5e01497435ba313bd561c7f57ca', run_uuid='0673d5e01497435ba313bd561c7f57ca', start_time=1651779137192, status='FINISHED', user_id='jovyan'>


### Registering a Model in the MLflow Model Registry

Model registration is also available in the web interface. To do this, select a model on the experiments page and click `Register Model`.

Below is a code that performs a similar action.

In [6]:
reg_model_name = "sk-learn-new-model"

# register model
client.create_registered_model(reg_model_name)
# create a new version
result = client.create_model_version(
    name=reg_model_name,
    source=f"{run_info.artifact_uri}/model",
    run_id=run_info.run_id
)

print(result)

2022/05/05 19:32:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-new-model, version 1


<ModelVersion: creation_timestamp=1651779144455, current_stage='None', description='', last_updated_timestamp=1651779144455, name='sk-learn-new-model', run_id='0673d5e01497435ba313bd561c7f57ca', run_link='', source='./mlruns/1/0673d5e01497435ba313bd561c7f57ca/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


View the list of registered models as follows:

In [7]:
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)

{   'creation_timestamp': 1651779144446,
    'description': '',
    'last_updated_timestamp': 1651779144455,
    'latest_versions': [   <ModelVersion: creation_timestamp=1651779144455, current_stage='None', description='', last_updated_timestamp=1651779144455, name='sk-learn-new-model', run_id='0673d5e01497435ba313bd561c7f57ca', run_link='', source='./mlruns/1/0673d5e01497435ba313bd561c7f57ca/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'sk-learn-new-model',
    'tags': {}}


You can find the model and version you need using a special function:

In [8]:
for mv in client.search_model_versions(f"name='{reg_model_name}'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1651779144455,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1651779144455,
    'name': 'sk-learn-new-model',
    'run_id': '0673d5e01497435ba313bd561c7f57ca',
    'run_link': '',
    'source': './mlruns/1/0673d5e01497435ba313bd561c7f57ca/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


## 3. Deployment of the model and testing the server

To use the model in a specific environment, you need to transfer the register the model to the desired environment. This operation only registers the model in the desired environment, does not start the web server.

### Deployment of the model in test and production environments

Terminology used in MLflow Model Registry:

  - Staging - test environment
  - Production - operational environment
 
The following code translates the previously registered version `1` into the` Staging` environment.

In [10]:
client.transition_model_version_stage(
    name=reg_model_name,
    version=1,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1651779144455, current_stage='Staging', description='', last_updated_timestamp=1651779177176, name='sk-learn-new-model', run_id='0673d5e01497435ba313bd561c7f57ca', run_link='', source='./mlruns/1/0673d5e01497435ba313bd561c7f57ca/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

Check the result with the following code:

In [11]:
for mv in client.search_model_versions(f"name='{reg_model_name}'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1651779144455,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1651779177176,
    'name': 'sk-learn-new-model',
    'run_id': '0673d5e01497435ba313bd561c7f57ca',
    'run_link': '',
    'source': './mlruns/1/0673d5e01497435ba313bd561c7f57ca/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}


To use the model, you need to start a web server, pass it the name of the model and environment, as well as the port (in the example, it runs on port 5005) as parameters.

To communicate with the web server, you must send requests to the `/invocations` entry point of that web server.

In [12]:
os.system('MLFLOW_TRACKING_URI=http://127.0.0.1:5000/ mlflow models serve -m "models:/sk-learn-new-model/Staging" -p 5005 --no-conda &')

0

In [14]:
import requests

url = f'http://127.0.0.1:5005/invocations'

http_data = test_x[:10].to_json(orient='split')
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: [5.88527611335971, 5.740523335804658, 5.811935039818762, 5.957850132893952, 6.092739574397946, 5.381768818172121, 6.050858460676247, 5.992469597297116, 5.850426199303677, 5.627508796953737]
